In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

In [9]:
from openpyxl.styles.colors import WHITE, RGB

__old_rgb_set__ = RGB.__set__


def __rgb_set_fixed__(self, instance, value):
    try:
        __old_rgb_set__(self, instance, value)
    except ValueError as e:
        if e.args[0] == 'Colors must be aRGB hex values':
            __old_rgb_set__(self, instance, WHITE)  # Change default color here


RGB.__set__ = __rgb_set_fixed__

In [19]:
df_aug = pd.read_excel("august_sale_date.xlsx", engine = 'openpyxl')

In [20]:
df_aug.head(5)

,SL,Outlet,Challan,Customer Name,Customer Code,Date,Item,Item Category,Item Sub Category,MRP,Sales Price,Total Quantity,Total Sales,Total Discount
0,1,EcoFab Limited,POSDLV0824838411,MD. KAMAL HOSSEN '[EF5462'],AP0370000781,2024-08-31,Speed Can 250ml [15554129],Food,Tiffin Item -2,50.0,49.0,3.0,147.0,3.0
1,2,Generation Next Fashions Ltd.,POSDLV0824838412,Md. Rasel '[D566-11-18'],AP0390001820,2024-08-31,Half Bowel Rice Premium [15552650],Food,Rice,62.0,59.6,25.0,1490.0,60.0
2,3,Generation Next Fashions Ltd.,POSDLV0824838412,Md. Rasel '[D566-11-18'],AP0390001820,2024-08-31,Lux Velvet Soft Touch Soap 100 Gm [15552845],Beauty and Health,Bath Soap,55.0,53.0,3.0,159.0,6.0
3,4,Generation Next Fashions Ltd.,POSDLV0824838412,Md. Rasel '[D566-11-18'],AP0390001820,2024-08-31,Mr.Noodles 28 Pcs [15552925],Food,Noodles,525.0,515.0,1.0,515.0,10.0
4,5,Interstoff Apparels Ltd.,POSDLV0824838413,Md. Shakil Shaikh '[133793'],AP0120402707,2024-08-31,Amore Family Cookies 300 Gm [15554152],Food,Family Pack,70.0,67.0,1.0,67.0,3.0


In [15]:
df_angel = pd.read_excel("Angel List - Spetember, Credit Upload.xlsx", engine = 'openpyxl')

In [17]:
df_angel.head(5)

,Customer Code,Contact Number,Outlet,Customer Name,HR Code,Gender,Joining Date,date,tenure_days
0,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284
1,AP0180000546,1736293994,Green Smart Shirts Ltd.,Shahina Khatun,2695,Male,2019-01-20 00:00:00,2024-08-31,2050
2,AP0180000570,1738139050,Green Smart Shirts Ltd.,Mst. Nargis Akter,2820,Male,2019-03-02 00:00:00,2024-08-31,2009
3,AP0180000726,1643973247,Green Smart Shirts Ltd.,Nasrin,3371,Male,2019-06-19 00:00:00,2024-08-31,1900
4,AP0180000790,1631834583,Green Smart Shirts Ltd.,Md. Masum Rana,3691,Male,2019-08-26 00:00:00,2024-08-31,1832


In [23]:
df_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120707 entries, 0 to 120706
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SL                 120707 non-null  int64  
 1   Outlet             120707 non-null  object 
 2   Challan            120707 non-null  object 
 3   Customer Name      120707 non-null  object 
 4   Customer Code      120707 non-null  object 
 5   Date               120707 non-null  object 
 6   Item               120707 non-null  object 
 7   Item Category      120707 non-null  object 
 8   Item Sub Category  120707 non-null  object 
 9   MRP                120707 non-null  float64
 10  Sales Price        120707 non-null  float64
 11  Total Quantity     120707 non-null  float64
 12  Total Sales        120707 non-null  float64
 13  Total Discount     120707 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 12.9+ MB


In [25]:
df_angel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5051 entries, 0 to 5050
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer Code   5051 non-null   object        
 1   Contact Number  4881 non-null   object        
 2   Outlet          5051 non-null   object        
 3   Customer Name   5051 non-null   object        
 4   HR Code         5051 non-null   object        
 5   Gender          3768 non-null   object        
 6   Joining Date    5051 non-null   object        
 7   date            5051 non-null   datetime64[ns]
 8   tenure_days     5051 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 355.3+ KB


In [27]:
# Merge the two dataframes on 'Customer Code'
merged_df = df_angel.merge(df_aug[['Customer Code', 'Date']], on='Customer Code', how='left')

In [33]:
# Convert 'Date' column to datetime in df_aug
df_aug['Date'] = pd.to_datetime(df_aug['Date'], errors='coerce')  # Using 'coerce' to handle any parsing errors

In [35]:
# Merge the two dataframes on 'Customer Code'
merged_df = df_angel.merge(df_aug[['Customer Code', 'Date']], on='Customer Code', how='left')

# Define a function to categorize the date into 'First 10 days' or 'Last 10 days'
def categorize_date(date):
    if pd.isna(date):
        return None
    elif date >= pd.Timestamp('2024-08-01') and date <= pd.Timestamp('2024-08-10'):
        return 'First 10 days'
    elif date >= pd.Timestamp('2024-08-11') and date <= pd.Timestamp('2024-08-20'):
        return 'Last 10 days'
    else:
        return None

In [37]:
# Apply the function to create the new 'State' column
merged_df['State'] = merged_df['Date'].apply(categorize_date)


In [39]:
merged_df.head(5)


,Customer Code,Contact Number,Outlet,Customer Name,HR Code,Gender,Joining Date,date,tenure_days,Date,State
0,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284,2024-08-07,First 10 days
1,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284,2024-08-07,First 10 days
2,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284,2024-08-07,First 10 days
3,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284,2024-08-07,First 10 days
4,AP0180000325,1927080126,Green Smart Shirts Ltd.,Onita,1730,Male,2018-05-31 00:00:00,2024-08-31,2284,2024-08-07,First 10 days


In [41]:
# Export the merged DataFrame to an Excel file
merged_df.to_excel('merged_df.xlsx', index=False)

print("Data exported to merged_df.xlsx successfully!")


Data exported to merged_df.xlsx successfully!


In [43]:
state_counts = merged_df['State'].value_counts()

# Display the counts
print(state_counts)

State
Last 10 days     24516
First 10 days    12724
Name: count, dtype: int64


In [47]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48401 entries, 0 to 48400
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer Code   48401 non-null  object        
 1   Contact Number  46923 non-null  object        
 2   Outlet          48401 non-null  object        
 3   Customer Name   48401 non-null  object        
 4   HR Code         48401 non-null  object        
 5   Gender          36146 non-null  object        
 6   Joining Date    48401 non-null  object        
 7   date            48401 non-null  datetime64[ns]
 8   tenure_days     48401 non-null  int64         
 9   Date            47923 non-null  datetime64[ns]
 10  State           37240 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 4.1+ MB


In [57]:
import pandas as pd

# Ensure 'Date' is in datetime format in df_aug
df_aug['Date'] = pd.to_datetime(df_aug['Date'], errors='coerce')

# Drop duplicates based on 'Customer Code' and keeping the earliest date
df_aug_unique = df_aug.sort_values('Date').drop_duplicates('Customer Code', keep='first')

# Merge with df_angel
merged_df = df_angel.merge(df_aug_unique[['Customer Code', 'Date']], on='Customer Code', how='left')

# Define the categorization function
def categorize_date(date):
    if pd.isna(date):
        return None
    elif pd.Timestamp('2024-08-01') <= date <= pd.Timestamp('2024-08-10'):
        return 'First 10 days'
    elif pd.Timestamp('2024-08-11') <= date <= pd.Timestamp('2024-08-31'):
        return 'Last 10 days'
    else:
        return None

# Apply the categorization
merged_df['State'] = merged_df['Date'].apply(categorize_date)


In [59]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5051 entries, 0 to 5050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Customer Code   5051 non-null   object        
 1   Contact Number  4881 non-null   object        
 2   Outlet          5051 non-null   object        
 3   Customer Name   5051 non-null   object        
 4   HR Code         5051 non-null   object        
 5   Gender          3768 non-null   object        
 6   Joining Date    5051 non-null   object        
 7   date            5051 non-null   datetime64[ns]
 8   tenure_days     5051 non-null   int64         
 9   Date            4573 non-null   datetime64[ns]
 10  State           4573 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 434.2+ KB


In [61]:
state_counts = merged_df['State'].value_counts()

# Display the counts
print(state_counts)

State
Last 10 days     2954
First 10 days    1619
Name: count, dtype: int64


In [65]:
# Export the merged DataFrame to an Excel file
merged_df.to_excel('angel_merged_df_2.xlsx', index=False)

print("Data exported to angel_merged_df.xlsx successfully!")


Data exported to angel_merged_df.xlsx successfully!
